# Model Training

### 1) Import Data and required packages

In [2]:
## Importing required liberaries and tools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Tools for model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
import warnings


In [3]:
## Reading dataset

df = pd.read_csv('data/student.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
## First we will create average score as our dependent feature

df['Average score'] = (df['math score'] + df['reading score'] + df['writing score'])/3
df.head()



,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,Average score
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333


In [5]:
## dEPENDENT AND INDEPENDENT FEATURES

X = df.drop('Average score', axis = 1)
y = df['Average score']

In [6]:
## Applying one hot encoding and standard scaler on independent features

num_features = X.select_dtypes(exclude = 'object').columns
cat_features = X.select_dtypes(include = 'object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
## Creating pipeline to perform tasks one after another
preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),  ## First perform one hot encoding on categorical features 
         ('StandardScaler', numeric_transformer, num_features), # Then perform standardization on nummeric features
    ]
)

In [7]:
X = preprocessor.fit_transform(X)

In [8]:
## Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

X_train.shape, X_test.shape

((800, 20), (200, 20))

### Creating a Evaluating function to give all metrics after model training. 


In [9]:
def evaluate_model(true, predicted):
    
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)

    return mae, rmse, r2_square

In [10]:
## With the help of list and for loop creating, training all models.
## Then predicting and evaluating models at a same time

models = {'LinearRegression':LinearRegression(),
          'Lasso':Lasso(),
          'Ridge':Ridge(),
          'K-Neighbour Regressor':KNeighborsRegressor(),
          'Decision Tree': DecisionTreeRegressor(),
          'Random Forest': RandomForestRegressor(),
          'XGBoost': XGBRegressor(),
          'Adaboost':AdaBoostRegressor()
          }

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Model Training
    
    ## Prediction making
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    
    ## Model Evaluation
    
    train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)
    
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])  ## Appending model keys in one list
    
    print('model Performance for training set')
    print("- Root mean squared error : {:.4f}".format(train_rmse))
    print('- Mean Absolute error: {:.4f}'.format(train_mae))
    print('- R2 score: {:.4f}'.format(train_r2))
    
    print('-----------------------------------')
    
    print('model performance for Test Set')
    print("- Root mean squared error : {:.4f}".format(test_rmse))
    print('- Mean Absolute error: {:.4f}'.format(test_mae))
    print('- R2 score: {:.4f}'.format(test_r2))
    
    r2_list.append(test_r2)
    
    print('='*35)
    print('\n')
    
    
    

LinearRegression
model Performance for training set
- Root mean squared error : 0.0000
- Mean Absolute error: 0.0000
- R2 score: 1.0000
-----------------------------------
model performance for Test Set
- Root mean squared error : 0.0000
- Mean Absolute error: 0.0000
- R2 score: 1.0000


Lasso
model Performance for training set
- Root mean squared error : 1.0645
- Mean Absolute error: 0.8506
- R2 score: 0.9943
-----------------------------------
model performance for Test Set
- Root mean squared error : 1.1142
- Mean Absolute error: 0.8769
- R2 score: 0.9942


Ridge
model Performance for training set
- Root mean squared error : 0.0080
- Mean Absolute error: 0.0065
- R2 score: 1.0000
-----------------------------------
model performance for Test Set
- Root mean squared error : 0.0088
- Mean Absolute error: 0.0067
- R2 score: 1.0000


K-Neighbour Regressor
model Performance for training set
- Root mean squared error : 2.2573
- Mean Absolute error: 1.7680
- R2 score: 0.9745
--------------

# Results


In [12]:
pd.DataFrame(list(zip(model_list, r2_list)), columns = ['Model_name', 'R2_score']).sort_values(by = ['R2_score'],ascending = False)

,Model_name,R2_score
0,LinearRegression,1.000000
2,Ridge,1.000000
6,XGBoost,0.994748
5,Random Forest,0.994345
1,Lasso,0.994209
4,Decision Tree,0.988405
7,Adaboost,0.979307
3,K-Neighbour Regressor,0.956016


Linear regression is giving best r2_score


Now, we will do all these steps in modular style coding.
